# Análise de Desempenho em Algoritmos de Aprendizado de Máquina Treinados em Dados com Viés

Renata Broder

Lilian Berton

## Configurações iniciais

Esta seção dedica-se à importação das bibliotecas necessárias para os experimentos que serão conduzidos.

### Pip install

In [ ]:
pip install aif360

In [ ]:
pip install 'aif360[AdversarialDebiasing]'

In [ ]:
pip install 'aif360[LFR]'

In [ ]:
pip install BlackBoxAuditing

### Imports

In [ ]:
%matplotlib inline
# Load all necessary packages
import sys
sys.path.append("../")
import numpy as np
import pandas as pd
from tqdm import tqdm

from aif360.datasets import BinaryLabelDataset, StandardDataset
from aif360.datasets import AdultDataset #, GermanDataset, CompasDataset

from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric
from aif360.metrics.utils import compute_boolean_conditioning_vector

from aif360.algorithms.preprocessing import Reweighing
from aif360.algorithms.preprocessing import DisparateImpactRemover

# from aif360.algorithms.preprocessing.optim_preproc import OptimPreproc
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions\
            import load_preproc_data_adult
# from aif360.algorithms.preprocessing.optim_preproc_helpers.distortion_functions\
#             import get_distortion_adult, get_distortion_german, get_distortion_compas
# from aif360.algorithms.preprocessing.optim_preproc_helpers.opt_tools import OptTools

from sklearn.calibration import CalibratedClassifierCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier

from IPython.display import Markdown, display
import matplotlib.pyplot as plt

pip install 'aif360[LFR]'


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Load do *Adult Dataset* e pré processamento inicial

Importar os arquivos do adult dataset do UCI para a pasta para a biblioteca aif360 poder realizar a leitura

In [ ]:
import requests

url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'
r = requests.get(url, allow_redirects=True)
open('/usr/local/lib/python3.6/dist-packages/aif360/data/raw/adult/adult.data', 'wb').write(r.content)

url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test'
r = requests.get(url, allow_redirects=True)
open('/usr/local/lib/python3.6/dist-packages/aif360/data/raw/adult/adult.test', 'wb').write(r.content)

url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names'
r = requests.get(url, allow_redirects=True)
open('/usr/local/lib/python3.6/dist-packages/aif360/data/raw/adult/adult.names', 'wb').write(r.content)

5229

Por dentro da função load_preproc_data_adult:

```python
load_preproc_data_adult(['sex']):

  XD_features = ['Age (decade)', 'Education Years', 'sex', 'race']
  D_features = ['sex']
  Y_features = ['Income Binary']
  X_features = list(set(XD_features)-set(D_features))
  categorical_features = ['Age (decade)', 'Education Years']

  # privileged classes
  all_privileged_classes = {"sex": [1.0],
                              "race": [1.0]}
  # protected attribute maps
  all_protected_attribute_maps = {"sex": {1.0: 'Male', 0.0: 'Female'},

  return AdultDataset(
        label_name=Y_features[0],
        favorable_classes=['>50K', '>50K.'],
        protected_attribute_names=D_features,
        privileged_classes=[all_privileged_classes[x] for x in D_features],
        instance_weights_name=None,
        categorical_features=categorical_features,
        features_to_keep=X_features+Y_features+D_features,
        na_values=['?'],
        metadata={'label_maps': [{1.0: '>50K', 0.0: '<=50K'}],
                  'protected_attribute_maps': [all_protected_attribute_maps[x]
                                for x in D_features]},
        custom_preprocessing=custom_preprocessing)
                                    "race": {1.0: 'White', 0.0: 'Non-white'}}

```


O custom_preprocessing é o pré processamento de dados que é realizado especialmente para o **Adult Dataset**:

```python
# Group age by decade
        df['Age (decade)'] = df['age'].apply(lambda x: x//10*10)

        def group_edu(x):
            if x <= 5: return '<6'
            elif x >= 13: return '>12'
            else: return x

        def age_cut(x):
            if x >= 70: return '>=70'
            else: return x

        def group_race(x):
            if x == "White": return 1.0
            else: return 0.0

        # Cluster education and age attributes.
        # Limit education range
        df['Education Years'] = df['education-num'].apply(lambda x: group_edu(x))
        df['Education Years'] = df['Education Years'].astype('category')

        # Limit age range
        df['Age (decade)'] = df['Age (decade)'].apply(lambda x: age_cut(x))

        # Rename income variable
        df['Income Binary'] = df['income-per-year']
        df['Income Binary'] = df['Income Binary'].replace(to_replace='>50K.', value='>50K', regex=True)
        df['Income Binary'] = df['Income Binary'].replace(to_replace='<=50K.', value='<=50K', regex=True)

        # Recode sex and race
        df['sex'] = df['sex'].replace({'Female': 0.0, 'Male': 1.0})
        df['race'] = df['race'].apply(lambda x: group_race(x))

```

  Além deste pré processamento customizado que consiste no agrupamento dos atributos 'Education Years', 'Age', além do tratamento das categorias dos atributos já mencionados, bem como dos atributos 'Sex' e 'Race' (passo 1), também é realizado um pré processamento na classe **StandardDataset** a partir dos passos seguintes:
2. Remoção de colunas não especificadas;
3. Remoção de linhas que contém Na
4. Tratamento para os dados categóricos
5. Binarização dos labels



## Classe desfavorecida: **Gênero e Raça**

In [ ]:
# Setar nome variável que seá utilizada no nome dos arquivos gerados
name_unprivileged_groups = 'Gênero e Raça' #'Gênero e Raça'

# Configurar gênero como classe desprivilegiada
privileged_groups = [{'sex': 1, 'race': 1}]#, {'sex': 1, 'race': 1}]
unprivileged_groups = [{'sex': 0, 'race': 0}]#, {'sex': 0, 'race': 0}]

list_unprivileged_groups = ['sex', 'race'] #'sex', 'race']

# Carregar o dataset já pré processado conforme processo descrito na seção anterior 
df_gender = load_preproc_data_adult(list_unprivileged_groups)

Agora a variável `df_gender` contém o dataset já pré processado. Vamos reutilizar este *dataset* para compor diferentes grupos de treino e teste gerados com diferentes *seeds* e medir a média da paridade demográfica 

In [ ]:
#random seed
np.random.seed(13)

# seeds = np.round((np.random.rand(30) * 1000))

seeds = [ 36, 184, 892, 707, 826, 715, 777, 251, 740, 134, 676, 708, 839, 343,
 393,  39, 937, 478, 325, 532, 161, 462, 660, 378, 872, 355, 636, 673,
 447,  34]
print(seeds)



[36, 184, 892, 707, 826, 715, 777, 251, 740, 134, 676, 708, 839, 343, 393, 39, 937, 478, 325, 532, 161, 462, 660, 378, 872, 355, 636, 673, 447, 34]


### Funções auxiliares

#### Métricas

In [ ]:
from collections import OrderedDict
from aif360.metrics import ClassificationMetric

def compute_metrics(dataset_true, dataset_pred, 
                    unprivileged_groups, privileged_groups,
                    best_class = -1,
                    disp = False):
    """ Compute the key metrics """
    m = ClassificationMetric(dataset_true,
                             dataset_pred, 
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)
    metrics = OrderedDict()
    metrics["Accuracy"] = m.accuracy()
    metrics["Balanced accuracy"] = 0.5*(m.true_positive_rate() + m.true_negative_rate())
    metrics["Precision"] = m.precision()
    metrics["Recall"] = m.recall()
    metrics["F1-Score"] = (2 * metrics["Precision"] * metrics["Recall"]) / (metrics["Precision"] + metrics["Recall"])
    metrics["Statistical parity difference"] = m.statistical_parity_difference()
    metrics["Disparate impact"] = m.disparate_impact()
    metrics["Average odds difference"] = m.average_odds_difference()
    metrics["Equal opportunity difference"] = m.equal_opportunity_difference()    
    metrics["Class Threshold"] = best_class
    
    if disp:
        for k in metrics:
            print("%s = %.4f" % (k, metrics[k]))
    
    return metrics

In [ ]:
def avg_metrics_seeds(metricas, name):
    # calculo das métricas para todos os testes
    m_bal_acc = []
    m_stat_par_diff = []
    m_disp_imp = []
    m_avg_odd_diff = []
    m_eq_opp_diff = []
    m_acc = []
    m_prec = []
    m_rec = []
    m_f1 = []


    df_metricas = pd.DataFrame.from_dict(metricas, orient='columns')

    for i, s in enumerate(metricas):
        m_acc.append(s['Accuracy'])
        m_avg_odd_diff.append(s['Average odds difference'])
        m_bal_acc.append(s['Balanced accuracy'])
        m_disp_imp.append(s['Disparate impact'])
        m_eq_opp_diff.append(s['Equal opportunity difference'])
        m_f1.append(s['F1-Score'])
        m_prec.append(s['Precision'])
        m_rec.append(s['Recall'])
        m_stat_par_diff.append(s['Statistical parity difference'])

    df_mean = pd.DataFrame({
        'Accuracy': round(np.mean(m_acc), 3),
        'Balanced accuracy': round(np.mean(m_bal_acc), 3),
        'Average odds difference': round(np.mean(m_avg_odd_diff), 3),
        'Balanced accuracy': round(np.mean(m_bal_acc), 3),
        'Disparate impact': round(np.mean(m_disp_imp), 3),
        'Equal opportunity difference': round(np.mean(m_eq_opp_diff), 3),
        'F1-Score': round(np.mean(m_f1), 3),
        'Precision': round(np.mean(m_prec), 3),
        'Recall': round(np.mean(m_rec), 3),
        'Statistical parity difference': round(np.mean(m_stat_par_diff), 3),
    }, index=['Media'])

    #df_aux = [np.mean(m_bal_acc), np.mean(m_avg_odd_diff), np.mean(m_bal_acc)]

    df_std = pd.DataFrame({
        'Accuracy': round(np.std(m_acc), 3),
        'Balanced accuracy': round(np.std(m_bal_acc), 3),
        'Average odds difference': round(np.std(m_avg_odd_diff), 3),
        'Balanced accuracy': round(np.std(m_bal_acc), 3),
        'Disparate impact': round(np.std(m_disp_imp), 3),
        'Equal opportunity difference': round(np.std(m_eq_opp_diff), 3),
        'F1-Score': round(np.std(m_f1), 3),
        'Precision': round(np.std(m_prec), 3),
        'Recall': round(np.std(m_rec), 3),
        'Statistical parity difference': round(np.std(m_stat_par_diff), 3),
    }, index=['DesvPad'])

    df_metricas = df_metricas.append(df_mean)
    df_metricas = df_metricas.append(df_std)

    df_metricas.to_csv(f'/content/drive/My Drive/UNIFESP/TCC/Arquivos - Adult/{name}.csv')

    metricas = {
        'ACC': np.mean(m_acc),
        'AOD': np.mean(m_avg_odd_diff),
        'BA': np.mean(m_bal_acc),
        'DI': np.mean(m_disp_imp),
        'EOD': np.mean(m_eq_opp_diff),
        'F1': np.mean(m_f1),
        'PRC': np.mean(m_prec),
        'REC': np.mean(m_rec),
        'SPD': np.mean(m_stat_par_diff),
    }

    display(Markdown("#### Acurácia (Objetivo: 1)"))
    print(metricas['ACC'])

    display(Markdown("#### Acurácia Balanceada (Objetivo: 1)"))
    print(metricas['BA'])

    display(Markdown("#### Diferença Paridade estatística (Objetivo: 0)"))
    print(metricas['SPD'])

    display(Markdown("#### Impacto diferenciado (Objetivo: 1)"))
    print(metricas['DI'])

    display(Markdown("#### F1 Score (Objetivo: 1)"))
    print(metricas['F1'])

    display(Markdown("#### Recall (Objetivo: 1)"))
    print(metricas['REC'])

    display(Markdown("#### Diferença de probabilidades (Objetivo: 0)"))
    print(metricas['AOD'])

    display(Markdown("#### Equal opportunities difference (Objetivo: 0)"))
    print(metricas['EOD'])

    return metricas

In [ ]:
def comparar_metricas(metricas, titulo, name):
    def autolabel(rects):
        """Attach a text label above each bar in *rects*, displaying its height."""
        for rect in rects:
            height = rect.get_height()
            ax.annotate('{}'.format(height),
                        xy=(rect.get_x() + rect.get_width() / 2, height),
                        xytext=(0, 3),  # 3 points vertical offset
                        textcoords="offset points",
                        ha='center', va='bottom')


    nome_metricas = ['Accuracy', 'Average Odds Difference', 'Balanced Accuracy', 'Disparate Impact', 'Equal Opportunities Difference', 'F1-Score', 'Precision', 'Recall', 'Statistical Parity Difference']
    objetivo =      [         1,                         0,                   1,                  1,                                0,          1,           1,        1,                               0]
    limites =       [  [0, 1.5],                   [-1, 1],            [0, 1.5],           [0, 1.5],                          [-1, 1],   [0, 1.5],    [0, 1.5], [0, 1.5],                         [-1, 1]]

    for i, m in enumerate(metricas[0][0]):
        #(i, m) = 0 BA, 1 SPD ...
        x = np.arange(len(metricas))  # the label locations
        width = 0.35  # the width of the bars

        fig, ax = plt.subplots(figsize=(6, 5))
        fig.patch.set_facecolor((1,1,1)) # color background

        ax.set_title(name) # Título do gráfico
        ax.set_xticks([]) # Definir que não tem ticks no eixo x
        ax.axhline(objetivo[i], label='Goal', color='k', linestyle=':')

        ax.set_xlabel(f'{nome_metricas[i]}')
        plt.ylim(limites[i][0], limites[i][1])

        for i, conjunto in enumerate(metricas):
            aux = ax.bar(x[i], round(conjunto[0][m], 3), width, label=conjunto[1])
            autolabel(aux)

        ax.legend()
        fig.tight_layout()
        # plt.show()
        fig.savefig(f'/content/drive/My Drive/UNIFESP/TCC/Gráficos - Adult/{name} - {titulo}_{m}.png') 
        plt.clf()

    



#### Treino

In [ ]:
def treino_valid_teste(df, clf, scaler, name, valid=True):
    melhor_acur_bal = np.zeros(30)
    melhor_lim_class = np.zeros(30)

    metricas = []
    metricas_seeds_sem_valid = []
    metricas_seeds = []

    for i, s in enumerate(seeds):
        np.random.seed(s)

        # Separação entre conjunto de teste, treino e validação
        df_train, df_vt = df.split([0.7], shuffle=True)
        df_valid, df_test = df_vt.split([0.5], shuffle=True)

        x_train = scaler.fit_transform(df_train.features) # escalarização do dataset original para treino
        y_train = df_train.labels.ravel() # label para o dataset de treino
        w_train = df_train.instance_weights.ravel() # pesos iguais = 1



        # Treino
        clf.fit(x_train,
                y_train,
                sample_weight=w_train) # treino com todos os pesos iguais a 1

        y_train_pred = clf.predict(x_train) # predição das mesmas entradas utilizadas para treino

        df_train_pred = df_train.copy()  
        df_train_pred.labels = y_train_pred # com os labels da predição feita com os dados de treino


        pos_ind = np.where(clf.classes_ == df_train.favorable_label)[0][0] # pega índice da classe favoravel

        # Validação
        df_valid_pred = df_valid.copy(deepcopy=True) # cria cópia do conjunto de validação
        x_valid = scaler.transform(df_valid_pred.features) # escalarização do conjunto
        y_valid = df_valid_pred.labels # armazena labels na variável y_valid
        df_valid_pred.scores = clf.predict_proba(x_valid)[:,pos_ind].reshape(-1,1) # guarda a probabilidade de a instância ser classificada como rotulo favorável (>50k)


        # Classification threshold "converte" a probabilidade em classes binárias

        # calcular acurácia balanceada e outras métricas para todos os limiares
        num_thresh = 100
        ba_arr = np.zeros(num_thresh)
        class_thresh_arr = np.linspace(0.01, 0.99, num_thresh)

        for idx, class_thresh in enumerate(class_thresh_arr):
            
            fav_inds = df_valid_pred.scores > class_thresh # Se a probabilidade de classificar a instancia de validação como favorita for maior do que o limiar de classe,
            df_valid_pred.labels[fav_inds] = df_valid_pred.favorable_label # classifica como favorável
            df_valid_pred.labels[~fav_inds] = df_valid_pred.unfavorable_label # se não, classifica como desfavorável
            
            metricas.append(ClassificationMetric(df_valid,       # dataset de validação original
                                                df_valid_pred,  # labels gerados pela probabilidade do classificador
                                                unprivileged_groups=unprivileged_groups,
                                                privileged_groups=privileged_groups))
            
            ba_arr[idx] = 0.5*(metricas[-1].true_positive_rate()\
                            +metricas[-1].true_negative_rate()) # calcula a média entre as taxas de verdadeiros positivos e negativos armazenando para cada limiar de classe

        best_ind = np.where(ba_arr == np.max(ba_arr))[0][0]
        best_class_thresh = class_thresh_arr[best_ind]

        melhor_acur_bal[i] = np.max(ba_arr)
        melhor_lim_class[i] = np.max(best_class_thresh)


        # Teste

        df_test_pred = df_test.copy(deepcopy=True) # cria cópia do conjunto de teste
        x_test = scaler.transform(df_test_pred.features) # escalarização do conjunto
        y_test = df_test_pred.labels # armazena labels na variável y_test
        df_test_pred.scores = clf.predict_proba(x_test)[:,pos_ind].reshape(-1,1) # guarda a probabilidade de a instância ser classificada como rotulo favorável (>50k)
            
        fav_inds = df_test_pred.scores > melhor_lim_class[i]
        df_test_pred.labels[fav_inds] = df_test_pred.favorable_label
        df_test_pred.labels[~fav_inds] = df_test_pred.unfavorable_label
        
        metric_test_bef = compute_metrics(df_test, df_test_pred, 
                                            unprivileged_groups, privileged_groups,
                                            best_class_thresh)
        metricas_seeds.append(metric_test_bef)



        # Teste sem valid
        df_test_pred.labels = clf.predict(x_test)
        metric_test_sem_valid = compute_metrics(df_test, df_test_pred, 
                                                unprivileged_groups, privileged_groups)
        metricas_seeds_sem_valid.append(metric_test_sem_valid)

    metrica_df_gender = avg_metrics_seeds(metricas_seeds, name)
    metrica_df_gender_sem_valid = avg_metrics_seeds(metricas_seeds_sem_valid, name + 'Sem validação')

    return metrica_df_gender, metrica_df_gender_sem_valid, melhor_lim_class

In [ ]:
def treino_RW(df, clf, scaler, bct, name):
    RW = Reweighing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)

    metricas_seeds_rw = []

    for i, s in enumerate(seeds):
        np.random.seed(s)

        # Separação entre conjunto de teste, treino e validação
        df_train, df_test = df.split([0.7], shuffle=True)
        # df_valid, df_test = df_vt.split([0.5], shuffle=True)

        # Rebalanceamento dos pesos
        df_train_rw = RW.fit_transform(df_train) # aplicado o reweight no conjunto de treino

        index = [df_train.feature_names.index(u) for u in list_unprivileged_groups]

        df_train_rw.features = scaler.fit_transform(df_train_rw.features)
        x_train_rw = df_train_rw.features#np.delete(df_train_rw.features, index, axis=1)
        y_train_rw = df_train_rw.labels.ravel()
        w_train_rw = df_train_rw.instance_weights

        # x_valid_rw = scaler.fit_transform(df_valid.features)
        #x_valid_rw = np.delete(x_valid_rw, index, axis=1)

        x_test_rw = scaler.fit_transform(df_test.features)
        #x_test_rw = np.delete(x_test_rw, index, axis=1)


        clf.fit(x_train_rw,
                y_train_rw, 
                sample_weight=w_train_rw) # treino com Regressão Logísitica e pesos do RW

        y_train_pred = clf.predict(x_train_rw) # predição das mesmas entradas utilizadas para treino

        pos_ind = np.where(clf.classes_ == df_train.favorable_label)[0][0] # pega índice da classe favoravel


        # Validação
        # df_valid_pred = df_valid.copy(deepcopy=True) # cria cópia do conjunto de validação
        # df_valid_pred.scores = clf.predict_proba(x_valid_rw)[:,pos_ind].reshape(-1,1) # guarda a probabilidade de a instância ser classificada como rotulo favorável (>50k)


        # Classification threshold "converte" a probabilidade em classes binárias

        # calcular acurácia balanceada e outras métricas para todos os limiares
        # num_thresh = 100
        # ba_arr = np.zeros(num_thresh)
        # class_thresh_arr = np.linspace(0.01, 0.99, num_thresh)
        # metricas = []

        # for idx, class_thresh in enumerate(class_thresh_arr):
            
        # fav_inds = df_valid_pred.scores > bct[i] # Se a probabilidade de classificar a instancia de validação como favorita for maior do que o limiar de classe,
        # df_valid_pred.labels[fav_inds] = df_valid_pred.favorable_label # classifica como favorável
        # df_valid_pred.labels[~fav_inds] = df_valid_pred.unfavorable_label # se não, classifica como desfavorável
        
        # metricas.append(ClassificationMetric(df_valid,       # dataset de validação original
        #                                     df_valid_pred,  # labels gerados pela probabilidade do classificador
        #                                     unprivileged_groups=unprivileged_groups,
        #                                     privileged_groups=privileged_groups))
        
        # ba_arr[idx] = 0.5*(metricas[-1].true_positive_rate()\
                        # +metricas[-1].true_negative_rate()) # calcula a média entre as taxas de verdadeiros positivos e negativos armazenando para cada limiar de classe

        # best_ind = np.where(ba_arr == np.max(ba_arr))[0][0]
        # best_class_thresh = class_thresh_arr[best_ind]


        # Teste
        df_rw_test_pred = df_test.copy(deepcopy=True) # cria cópia do conjunto de teste
        df_rw_test_pred.scores = clf.predict_proba(x_test_rw)[:,pos_ind].reshape(-1,1) # guarda a probabilidade de a instância ser classificada como rotulo favorável (>50k)
                    
        fav_inds = df_rw_test_pred.scores > bct[i]
        df_rw_test_pred.labels[fav_inds] = df_rw_test_pred.favorable_label
        df_rw_test_pred.labels[~fav_inds] = df_rw_test_pred.unfavorable_label
        
        metric_test_aft = compute_metrics(df_test, df_rw_test_pred, 
                                          unprivileged_groups, privileged_groups,
                                          best_class=bct[i],
                                          disp = False)
        metricas_seeds_rw.append(metric_test_aft)

    metrica_df_rw = avg_metrics_seeds(metricas_seeds_rw, name)

    return metrica_df_rw


In [ ]:
def treino_DIR(df, clf, scaler, bct, name):
    di = DisparateImpactRemover()

    metric_df_di = []
    for i, s in enumerate(seeds):
        np.random.seed(s)
        # Separação entre conjunto de teste, treino e validação
        df_train, df_test = df.split([0.7], shuffle=True)
        # df_valid, df_test = df_vt.split([0.5], shuffle=True)

        df_train.features = scaler.fit_transform(df_train.features)
        # df_valid.features = scaler.fit_transform(df_valid.features)
        df_test.features = scaler.fit_transform(df_test.features)

        index = [df_train.feature_names.index(u) for u in list_unprivileged_groups]

        df_train_dir = di.fit_transform(df_train)
        x_train_dir = np.delete(df_train_dir.features, index, axis=1)
        y_train_dir = df_train_dir.labels.ravel()
        
        # df_valid_dir = di.fit_transform(df_valid)
        # x_valid_dir = np.delete(df_valid_dir.features, index, axis=1)

        df_test_dir = di.fit_transform(df_test)
        x_test_dir = np.delete(df_test_dir.features, index, axis=1)
        
        clf.fit(x_train_dir, y_train_dir)

        pos_ind = np.where(clf.classes_ == df_test.favorable_label)[0][0] # pega índice da classe favoravel

        # # Validação
        # df_valid_pred = df_valid_dir.copy(deepcopy=True) # cria cópia do conjunto de validação
        # x_valid = scaler.transform(df_valid_pred.features) # escalarização do conjunto
        # y_valid = df_valid_pred.labels # armazena labels na variável y_valid
        # df_valid_pred.scores = clf.predict_proba(x_valid_dir)[:,pos_ind].reshape(-1,1) # guarda a probabilidade de a instância ser classificada como rotulo favorável (>50k)


        # Classification threshold "converte" a probabilidade em classes binárias

        # # calcular acurácia balanceada e outras métricas para todos os limiares
        # num_thresh = 100
        # ba_arr = np.zeros(num_thresh)
        # class_thresh_arr = np.linspace(0.01, 0.99, num_thresh)
        # metricas = []

        # for idx, class_thresh in enumerate(class_thresh_arr):
            
        #     fav_inds = df_valid_pred.scores > class_thresh # Se a probabilidade de classificar a instancia de validação como favorita for maior do que o limiar de classe,
        #     df_valid_pred.labels[fav_inds] = df_valid_pred.favorable_label # classifica como favorável
        #     df_valid_pred.labels[~fav_inds] = df_valid_pred.unfavorable_label # se não, classifica como desfavorável
            
        #     metricas.append(ClassificationMetric(df_valid,       # dataset de validação original
        #                                         df_valid_pred,  # labels gerados pela probabilidade do classificador
        #                                         unprivileged_groups=unprivileged_groups,
        #                                         privileged_groups=privileged_groups))
            
        #     ba_arr[idx] = 0.5*(metricas[-1].true_positive_rate()\
        #                     +metricas[-1].true_negative_rate()) # calcula a média entre as taxas de verdadeiros positivos e negativos armazenando para cada limiar de classe

        # best_ind = np.where(ba_arr == np.max(ba_arr))[0][0]
        # best_class_thresh = class_thresh_arr[best_ind]


        # Treino

        df_test_pred = df_test.copy(deepcopy=True)
        df_test_pred.scores = clf.predict_proba(x_test_dir)[:,pos_ind].reshape(-1,1) # guarda a probabilidade de a instância ser classificada como rotulo favorável (>50k)

        fav_inds = df_test_pred.scores > bct[i]
        df_test_pred.labels[fav_inds] = df_test_pred.favorable_label
        df_test_pred.labels[~fav_inds] = df_test_pred.unfavorable_label

        ##

        metric_test_aft = compute_metrics(df_test, df_test_pred, 
                                            unprivileged_groups, privileged_groups,
                                            best_class=bct[i],
                                            disp = False)
                
        # Métrica (conjunto de treino): Statistical Parity Difference
        metric_df_di.append(metric_test_aft)
    
    metrica_df_dir = avg_metrics_seeds(metric_df_di, name)

    return metrica_df_dir

## Técnicas de pré processamento para mitigação de viés

#### **Paridade demográfica (*Statistical Parity Difference - SPD*)**

A distribuição da probabilidade de predição positiva (ou negativa) deve ser idêntica nas subpopulações privilegiadas e desprivilegiadas:

$$ SPD = 𝑃𝑟(𝑌=1|𝐷=unprivileged)−𝑃𝑟(𝑌=1|𝐷=privileged) $$

SPD é "mais justo" quanto mais próximo de 0!






In [ ]:
metric_df_gender_spd = []

for s in seeds:
    np.random.seed(s)
    # Separação entre conjunto de teste, treino e validação
    df_gender_train, df_gender_vt = df_gender.split([0.7], shuffle=True)
    #df_gender_valid, df_gender_test = df_gender_vt.split([0.5], shuffle=True)

    # Métrica (conjunto de treino): Statistical Parity Difference
    metric_df_gender_spd.append(BinaryLabelDatasetMetric(df_gender_train, 
                                                         unprivileged_groups=unprivileged_groups,
                                                         privileged_groups=privileged_groups)
                                                        .mean_difference())


display(Markdown("#### SPD - Adult Dataset Original (grupo discriminado: gênero)"))
print("Média da diferença nos resultados médios entre grupos não privilegiados e privilegiados = %f" % np.mean(metric_df_gender_spd))
print("Desvio padrão da diferença nos resultados médios entre grupos não privilegiados e privilegiados = %f" % np.std(metric_df_gender_spd))

#### SPD - Adult Dataset Original (grupo discriminado: gênero)

Média da diferença nos resultados médios entre grupos não privilegiados e privilegiados = -0.243302
Desvio padrão da diferença nos resultados médios entre grupos não privilegiados e privilegiados = 0.003876


In [ ]:
metric_df_gender_spd_rw = []
RW = Reweighing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)

for s in seeds:
  np.random.seed(s)
  # Separação entre conjunto de teste, treino e validação
  df_gender_train, df_gender_vt = df_gender.split([0.7], shuffle=True)
  #df_gender_valid, df_gender_test = df_gender_vt.split([0.5], shuffle=True)
  
  df_gender_rw_train = RW.fit_transform(df_gender_train) # aplicado o reweight no conjunto de treino

  metric_df_gender_spd_rw.append(BinaryLabelDatasetMetric(df_gender_rw_train, 
                                                          unprivileged_groups=unprivileged_groups,
                                                          privileged_groups=privileged_groups)
                                                          .mean_difference())
  
display(Markdown("#### SPD - Adult Dataset Rebalanceado (grupo discriminado: gênero)"))
print("Média da diferença nos resultados médios entre grupos não privilegiados e privilegiados = %f" % np.mean(metric_df_gender_spd_rw))
print("Desvio padrão da diferença nos resultados médios entre grupos não privilegiados e privilegiados = %f" % np.std(metric_df_gender_spd_rw))

#### SPD - Adult Dataset Rebalanceado (grupo discriminado: gênero)

Média da diferença nos resultados médios entre grupos não privilegiados e privilegiados = -0.000000
Desvio padrão da diferença nos resultados médios entre grupos não privilegiados e privilegiados = 0.000000


#### Regressão Logística

##### Dataset original

In [ ]:
scale_orig = StandardScaler()
lmod = LogisticRegression(class_weight='balanced', solver='liblinear')

metrica_RL, metrica_RL_sem_valid, threshold_RL = treino_valid_teste(df_gender, lmod, scale_orig, f'{name_unprivileged_groups} - RL - Original')


#### Acurácia (Objetivo: 1)

0.7374368773031253


#### Acurácia Balanceada (Objetivo: 1)

0.7441653519690381


#### Diferença Paridade estatística (Objetivo: 0)

...

0.7714717458211591


#### Diferença de probabilidades (Objetivo: 0)

-0.44993134947527014


#### Equal opportunities difference (Objetivo: 0)

-0.5287196726967659


##### Dataset Reweighing

In [ ]:
scale_transf = StandardScaler()
lmod = LogisticRegression(class_weight='balanced', solver='liblinear')

metrica_RL_RW = treino_RW(df_gender, lmod, scale_transf, threshold_RL, f'{name_unprivileged_groups} - RL - Reweighing')


#### Acurácia (Objetivo: 1)

0.7622420892194999


#### Acurácia Balanceada (Objetivo: 1)

0.7223208893518213


#### Diferença Paridade estatística (Objetivo: 0)

...

0.6457032215272496


#### Diferença de probabilidades (Objetivo: 0)

-0.09289926415482573


#### Equal opportunities difference (Objetivo: 0)

-0.09695784624577719


##### Dataset Disparate Impact Remover

In [ ]:
scaler_orig = MinMaxScaler() #MinMaxScaler()StandardScaler()
lmod = LogisticRegression(class_weight='balanced', solver='liblinear')

metrica_RL_DIR = treino_DIR(df_gender, lmod, scaler_orig, threshold_RL, f'{name_unprivileged_groups} - RL - Disparate Impact Remover')

#### Acurácia (Objetivo: 1)

0.7232375622739372


#### Acurácia Balanceada (Objetivo: 1)

0.713940190093824


#### Diferença Paridade estatística (Objetivo: 0)

...

0.6961062627820505


#### Diferença de probabilidades (Objetivo: 0)

-0.0070329827784765975


#### Equal opportunities difference (Objetivo: 0)

-0.012825152449152544


##### Comparativo

In [ ]:
comparar_metricas([[metrica_RL, 'Original'],
                   [metrica_RL_RW, 'Reweighing'],
                   [metrica_RL_DIR, 'Disparate Impact Remover']],
                  'Regressão Logística', f'{name_unprivileged_groups} - RL')

#### Random Forest

##### Dataset original

In [ ]:
scale_orig = StandardScaler()
RF = RandomForestClassifier()

metrica_RF, metrica_RF_sem_valid, threshold_RF = treino_valid_teste(df_gender, RF, scale_orig, f'{name_unprivileged_groups} - RF - Original')


#### Acurácia (Objetivo: 1)

0.7341294754560758


#### Acurácia Balanceada (Objetivo: 1)

0.7436358221415863


#### Diferença Paridade estatística (Objetivo: 0)

...

0.3938562458006987


#### Diferença de probabilidades (Objetivo: 0)

-0.29845886524034526


#### Equal opportunities difference (Objetivo: 0)

-0.4766578506925923


##### Dataset Reweighing

In [ ]:
scale_transf = StandardScaler()
RW = Reweighing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
RF = RandomForestClassifier()

metrica_RF_RW = treino_RW(df_gender, RF, scale_transf, threshold_RF, f'{name_unprivileged_groups} - RF - Reweighing')


#### Acurácia (Objetivo: 1)

0.7583179781159719


#### Acurácia Balanceada (Objetivo: 1)

0.7263412163267197


#### Diferença Paridade estatística (Objetivo: 0)

...

0.6651260450176787


#### Diferença de probabilidades (Objetivo: 0)

-0.08171862877665619


#### Equal opportunities difference (Objetivo: 0)

-0.11276241638789025


##### Dataset Disparate Impact Remover

In [ ]:
scaler_orig = MinMaxScaler()
RF = RandomForestClassifier()

metrica_RF_DIR = treino_DIR(df_gender, RF, scaler_orig, threshold_RF, f'{name_unprivileged_groups} - RF - Disparate Impact Remover')

#### Acurácia (Objetivo: 1)

0.7177574558110965


#### Acurácia Balanceada (Objetivo: 1)

0.713462981683551


#### Diferença Paridade estatística (Objetivo: 0)

...

0.7054852430021936


#### Diferença de probabilidades (Objetivo: 0)

-0.008404511089933913


#### Equal opportunities difference (Objetivo: 0)

-0.020269362353599867


##### Comparativo

In [ ]:
comparar_metricas([[metrica_RF, 'Original'],
                   [metrica_RF_RW, 'Reweighing'],
                   [metrica_RF_DIR, 'Disparate Impact Remover']],
                  'Random Forest', f'{name_unprivileged_groups} - RF')

#### Nayve Bayes Bernoulli

##### Dataset original

In [ ]:
scale_orig = StandardScaler()
NB = BernoulliNB()

metrica_NB, metrica_NB_sem_valid, threshold_NB = treino_valid_teste(df_gender, NB, scale_orig, f'{name_unprivileged_groups} - NB - Original')


#### Acurácia (Objetivo: 1)

0.7384604886037943


#### Acurácia Balanceada (Objetivo: 1)

0.7431045351718037


#### Diferença Paridade estatística (Objetivo: 0)

...

0.4554536951104361


#### Diferença de probabilidades (Objetivo: 0)

-0.3155898562408638


#### Equal opportunities difference (Objetivo: 0)

-0.4986855515943771


##### Dataset Reweighing

In [ ]:
scale_transf = StandardScaler()
NB = BernoulliNB()

metrica_NB_RW = treino_RW(df_gender, NB, scale_transf, threshold_NB, f'{name_unprivileged_groups} - NB - Reweighing')

#### Acurácia (Objetivo: 1)

0.7655747400987284


#### Acurácia Balanceada (Objetivo: 1)

0.7176554308523688


#### Diferença Paridade estatística (Objetivo: 0)

...

0.6257002727454263


#### Diferença de probabilidades (Objetivo: 0)

-0.10170795093288564


#### Equal opportunities difference (Objetivo: 0)

-0.10552862917371625


##### Dataset Disparate Impact Remover

In [ ]:
scaler_orig = MinMaxScaler()
NB = BernoulliNB()

metrica_NB_DIR = treino_DIR(df_gender, NB, scaler_orig, threshold_NB, f'{name_unprivileged_groups} - NB - Disparate Impact Remover')

#### Acurácia (Objetivo: 1)

0.7354261925885484


#### Acurácia Balanceada (Objetivo: 1)

0.7102662141235502


#### Diferença Paridade estatística (Objetivo: 0)

...

0.6620067467877947


#### Diferença de probabilidades (Objetivo: 0)

0.014643867995042545


#### Equal opportunities difference (Objetivo: 0)

0.02337280857062961


##### Comparativo

In [ ]:
comparar_metricas([[metrica_NB, 'Original'],
                   [metrica_NB_RW, 'Reweighing'],
                   [metrica_NB_DIR, 'Disparate Impact Remover']],
                  'Naive Bayes', f'{name_unprivileged_groups} - NB')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


#### Árvore de decisão

##### Dataset original

In [ ]:
scale_orig = StandardScaler()
DT = DecisionTreeClassifier()

metrica_DT, metrica_DT_sem_valid, threshold_DT = treino_valid_teste(df_gender, DT, scale_orig, f'{name_unprivileged_groups} - AD - Original')


#### Acurácia (Objetivo: 1)

0.734193166825895


#### Acurácia Balanceada (Objetivo: 1)

0.7435872440644343


#### Diferença Paridade estatística (Objetivo: 0)

...

0.39197384253459056


#### Diferença de probabilidades (Objetivo: 0)

-0.29670596921210646


#### Equal opportunities difference (Objetivo: 0)

-0.47438018424410344


##### Dataset  Reweighing

In [ ]:
scale_transf = StandardScaler()
DT = DecisionTreeClassifier()

metrica_DT_RW = treino_RW(df_gender, DT, scale_transf, threshold_DT, f'{name_unprivileged_groups} - AD - Reweighing')

#### Acurácia (Objetivo: 1)

0.757888032029846


#### Acurácia Balanceada (Objetivo: 1)

0.7256521488543338


#### Diferença Paridade estatística (Objetivo: 0)

...

0.664045708235637


#### Diferença de probabilidades (Objetivo: 0)

-0.07303356187417588


#### Equal opportunities difference (Objetivo: 0)

-0.10140955408595254


##### Dataset Disparate Impact Remover

In [ ]:
scaler_orig = MinMaxScaler()
DT = DecisionTreeClassifier()

metrica_DT_DIR = treino_DIR(df_gender, DT, scaler_orig, threshold_DT, f'{name_unprivileged_groups} - AD - Disparate Impact Remover')

#### Acurácia (Objetivo: 1)

0.7182692963898176


#### Acurácia Balanceada (Objetivo: 1)

0.7129072142895654


#### Diferença Paridade estatística (Objetivo: 0)

...

0.7029405503012275


#### Diferença de probabilidades (Objetivo: 0)

-0.008352862350405511


#### Equal opportunities difference (Objetivo: 0)

-0.019453776748766416


##### Comparativo

In [ ]:
comparar_metricas([[metrica_DT, 'Original'],
                   [metrica_DT_RW, 'Reweighing'],
                   [metrica_DT_DIR,'Disparate Impact Remover']],
                  'Árvore de decisão', f'{name_unprivileged_groups} - AD')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


#### SVM

##### Dataset original

In [ ]:
scale_orig = StandardScaler()
LinearSVM = LinearSVC(dual=False)
SVM = CalibratedClassifierCV(LinearSVM)

metrica_SVM, metrica_SVM_sem_valid, threshold_SVM = treino_valid_teste(df_gender, SVM, scale_orig, f'{name_unprivileged_groups} - SVM - Original')


#### Acurácia (Objetivo: 1)

0.7281879805286383


#### Acurácia Balanceada (Objetivo: 1)

0.7405890052101921


#### Diferença Paridade estatística (Objetivo: 0)

...

0.3888782352033942


#### Diferença de probabilidades (Objetivo: 0)

-0.2918050266403925


#### Equal opportunities difference (Objetivo: 0)

-0.46834130409576147


##### Dataset Reweighing

In [ ]:
scale_transf = StandardScaler()
LinearSVM = LinearSVC(dual=False)
SVM = CalibratedClassifierCV(LinearSVM)

metrica_SVM_RW = treino_RW(df_gender, SVM, scale_transf, threshold_SVM, f'{name_unprivileged_groups} - SVM - Reweighing')

#### Acurácia (Objetivo: 1)

0.7627744034213698


#### Acurácia Balanceada (Objetivo: 1)

0.7183640538441725


#### Diferença Paridade estatística (Objetivo: 0)

...

0.633182914349808


#### Diferença de probabilidades (Objetivo: 0)

-0.09604138746285527


#### Equal opportunities difference (Objetivo: 0)

-0.09965830166504593


##### Dataset transformado pelo Disparate Impact Remover

In [ ]:
scaler_orig = MinMaxScaler()
LinearSVM = LinearSVC(dual=False)
SVM = CalibratedClassifierCV(LinearSVM)

metrica_SVM_DIR = treino_DIR(df_gender, SVM, scaler_orig, threshold_SVM, f'{name_unprivileged_groups} - SVM - Disparate Impact Remover')

#### Acurácia (Objetivo: 1)

0.7340567346845924


#### Acurácia Balanceada (Objetivo: 1)

0.7104581551758941


#### Diferença Paridade estatística (Objetivo: 0)

...

0.6652605708238444


#### Diferença de probabilidades (Objetivo: 0)

-0.004045845193772703


#### Equal opportunities difference (Objetivo: 0)

-0.005964705841760057


##### Comparativo

In [ ]:
comparar_metricas([[metrica_SVM, 'Original'],
                   [metrica_SVM_RW, 'Reweighing'],
                   [metrica_SVM_DIR, 'Disparate Impact Remover']],
                  'SVM', f'{name_unprivileged_groups} - SVM') #[metrica_SVM_DIR,'Disparate Impact Remover']], 'SVM')
                  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


#### MLP

##### Dataset original

In [ ]:
scale_orig = StandardScaler()
MLP_cl = MLPClassifier()
MLP = CalibratedClassifierCV(MLP_cl)

metrica_MLP, metrica_MLP_sem_valid, threshold_MLP = treino_valid_teste(df_gender, MLP, scale_orig, f'{name_unprivileged_groups} - MLP - Original')


/usr/local/lib/python3.6/dist-packages/sklearn/calibration.py:175: UserWarning: MLPClassifier does not support sample_weight. Samples weights are only used for the calibration itself.
  " itself." % estimator_name)
/usr/local/lib/python3.6/dist-packages/sklearn/calibration.py:175: UserWarning: MLPClassifier does not support sample_weight. Samples weights are only used for the calibration itself.
  " itself." % estimator_name)
/usr/local/lib/python3.6/dist-packages/sklearn/calibration.py:175: UserWarning: MLPClassifier does not support sample_weight. Samples weights are only used for the calibration itself.
  " itself." % estimator_name)
/usr/local/lib/python3.6/dist-packages/sklearn/calibration.py:175: UserWarning: MLPClassifier does not support sample_weight. Samples weights are only used for the calibration itself.
  " itself." % estimator_name)
/usr/local/lib/python3.6/dist-packages/sklearn/calibration.py:175: UserWarning: MLPClassifier does not support sample_weight. Samples weight

#### Acurácia (Objetivo: 1)

0.7336927346344572


#### Acurácia Balanceada (Objetivo: 1)

0.743847964839656


...

0.39104346937677464


#### Diferença de probabilidades (Objetivo: 0)

-0.29633278737004976


#### Equal opportunities difference (Objetivo: 0)

-0.4738638607212807


##### Dataset Reweighing

In [ ]:
scale_transf = StandardScaler()
MLP_cl = MLPClassifier()
MLP = CalibratedClassifierCV(MLP_cl)

metrica_MLP_RW = treino_RW(df_gender, MLP, scale_transf, threshold_MLP, f'{name_unprivileged_groups} - MLP - Reweighing')

/usr/local/lib/python3.6/dist-packages/sklearn/calibration.py:175: UserWarning: MLPClassifier does not support sample_weight. Samples weights are only used for the calibration itself.
  " itself." % estimator_name)
/usr/local/lib/python3.6/dist-packages/sklearn/calibration.py:175: UserWarning: MLPClassifier does not support sample_weight. Samples weights are only used for the calibration itself.
  " itself." % estimator_name)
/usr/local/lib/python3.6/dist-packages/sklearn/calibration.py:175: UserWarning: MLPClassifier does not support sample_weight. Samples weights are only used for the calibration itself.
  " itself." % estimator_name)
/usr/local/lib/python3.6/dist-packages/sklearn/calibration.py:175: UserWarning: MLPClassifier does not support sample_weight. Samples weights are only used for the calibration itself.
  " itself." % estimator_name)
/usr/local/lib/python3.6/dist-packages/sklearn/calibration.py:175: UserWarning: MLPClassifier does not support sample_weight. Samples weight

#### Acurácia (Objetivo: 1)

0.7513410223162492


#### Acurácia Balanceada (Objetivo: 1)

0.7393216385309832


...

0.7163345389717957


#### Diferença de probabilidades (Objetivo: 0)

-0.41262533780270066


#### Equal opportunities difference (Objetivo: 0)

-0.5208193089604618


##### Dataset transformado pelo Disparate Impact Remover

In [ ]:
scaler_orig = MinMaxScaler()
MLP_cl = MLPClassifier()
MLP = CalibratedClassifierCV(MLP_cl)

metrica_MLP_DIR = treino_DIR(df_gender, MLP, scaler_orig, threshold_MLP, f'{name_unprivileged_groups} - MLP - Disparate Impact Remover')

#### Acurácia (Objetivo: 1)

0.7112809663550125


#### Acurácia Balanceada (Objetivo: 1)

0.713215789384505


#### Diferença Paridade estatística (Objetivo: 0)

...

0.7170199153509146


#### Diferença de probabilidades (Objetivo: 0)

-0.004821580002863747


#### Equal opportunities difference (Objetivo: 0)

-0.015070781461616537


##### Comparativo

In [ ]:
comparar_metricas([[metrica_MLP, 'Original'],
                   [metrica_MLP_RW, 'Reweighing'],
                   [metrica_MLP_DIR,'Disparate Impact Remover']],
                  'MLP', f'{name_unprivileged_groups} - MLP')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


### Comparativo entre todos os algoritmos rodando no dataset original

In [ ]:
comparar_metricas([[metrica_RL, 'Regressão Logística'],
                   [metrica_RF, 'Random Forest'],
                   [metrica_NB, 'Nayve Bayes'],
                   [metrica_DT, 'Árvore de Decisão'],
                   [metrica_SVM, 'SVM'],
                   [metrica_MLP, 'MLP']],
                   'Algoritmos', f'{name_unprivileged_groups} - Sem debiasing')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


### Comparativo entre todos os algoritmos rodando no dataset original - Sem validação

In [ ]:
comparar_metricas([[metrica_RL_sem_valid, 'Regressão Logística'],
                   [metrica_RF_sem_valid, 'Random Forest'],
                   [metrica_NB_sem_valid, 'Nayve Bayes'],
                   [metrica_DT_sem_valid, 'Árvore de Decisão'],
                   [metrica_SVM_sem_valid, 'SVM'],
                   [metrica_MLP_sem_valid, 'MLP']],
                   'Algoritmos', f'{name_unprivileged_groups} - Sem debiasing - Sem Validação')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


### Comparativo entre todos os algoritmos utilizando Reweighing

In [ ]:
comparar_metricas([[metrica_RL_RW, 'Regressão Logística'],
                   [metrica_RF_RW, 'Random Forest'],
                   [metrica_NB_RW, 'Nayve Bayes'],
                   [metrica_DT_RW, 'Árvore de Decisão'],
                   [metrica_SVM_RW, 'SVM'],
                   [metrica_MLP_RW, 'MLP']],
                   'Algoritmos', f'{name_unprivileged_groups} - Reweighing')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


### Comparativo entre todos os algoritmos utilizando Disparate Impact Remover

In [ ]:
comparar_metricas([[metrica_RL_DIR, 'Regressão Logística'],
                   [metrica_RF_DIR, 'Random Forest'],
                   [metrica_NB_DIR, 'Nayve Bayes'],
                   [metrica_DT_DIR, 'Árvore de Decisão'],
                   [metrica_SVM_DIR, 'SVM'],
                   [metrica_MLP_DIR, 'MLP']],
                   'Algoritmos', f'{name_unprivileged_groups} - Disparate Impact Remover')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
